# Assignment 6: Predictive Modeling of Housing Prices in Philadelphia

**Due date: Wednesday, 12/6 by the end of the day**


Lectures 12B and 13A will cover predictive modeling of housing prices in Philadelphia. We'll extend that analysis in this section by:

- Optimizing our hyperparameters during the modeling process using cross-validation and a grid search
- Testing the fairness of our model by calculating the intersection of the model error rate and poverty rate across neighborhoods

## Part 2: Modeling Philadelphia's Housing Prices and Algorithmic Fairness


### 2.1 Load data from the Office of Property Assessment

Use the requests package to query the CARTO API for **single-family** property assessment data in Philadelphia for properties that had their **last sale during 2022**.

Sources: 
- [OpenDataPhilly](https://www.opendataphilly.org/dataset/opa-property-assessments)
- [Metadata](http://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/)

In [21]:
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import requests

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Show all columns
pd.options.display.max_columns = 999

In [2]:
# The API endpoint
carto_api_endpoint = "https://phl.carto.com/api/v2/sql"

# The query parameters
params = {
    "q": "SELECT * FROM opa_properties_public WHERE category_code_description = 'SINGLE FAMILY' AND sale_date >= '2022-01-01' AND sale_date < '2022-12-31'",
    "format": "geojson",
    "skipfields": "cartodb_id"
}

response = requests.get(carto_api_endpoint, params=params)

response

<Response [200]>

In [3]:
features = response.json()

In [4]:
homes = gpd.GeoDataFrame.from_features(features, crs="EPSG:4326")
homes_clean = homes.dropna(subset=['geometry'])

### 2.2 Load data for census tracts and neighborhoods

Load various Philadelphia-based regions that we will use in our analysis. 

- Census tracts can be downloaded from: [https://opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson](https://opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson)
- Neighborhoods can be downloaded from:
[https://raw.githubusercontent.com/azavea/geo-data/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson](https://raw.githubusercontent.com/azavea/geo-data/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson)


In [5]:
neigh = gpd.read_file("https://raw.githubusercontent.com/azavea/geo-data/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson")
tract = gpd.read_file("Census_Tracts_2010.geojson")

In [6]:
type(neigh)

geopandas.geodataframe.GeoDataFrame

In [7]:
tract.head()

,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,LOGRECNO,geometry
0,1,42,101,009400,42101009400,94,Census Tract 94,G5020,S,366717,0,+39.9632709,-075.2322437,10429,"POLYGON ((-75.22927 39.96054, -75.22865 39.960..."
1,2,42,101,009500,42101009500,95,Census Tract 95,G5020,S,319070,0,+39.9658709,-075.2379140,10430,"POLYGON ((-75.23536 39.96852, -75.23545 39.969..."
2,3,42,101,009600,42101009600,96,Census Tract 96,G5020,S,405273,0,+39.9655396,-075.2435075,10431,"POLYGON ((-75.24343 39.96230, -75.24339 39.962..."
3,4,42,101,013800,42101013800,138,Census Tract 138,G5020,S,341256,0,+39.9764504,-075.1771771,10468,"POLYGON ((-75.17341 39.97779, -75.17386 39.977..."
4,5,42,101,013900,42101013900,139,Census Tract 139,G5020,S,562934,0,+39.9750563,-075.1711846,10469,"POLYGON ((-75.17313 39.97776, -75.17321 39.977..."


### 2.3 Spatially join the sales data and neighborhoods/census tracts.

Perform a spatial join, such that each sale has an associated neighborhood and census tract.

**Note:** After performing the first spatial join, you will need to use the `drop()` function to remove the `index_right` column; otherwise an error will be raised on the second spatial join about duplicate columns.

In [14]:
neightracts = tract.sjoin(neigh, how='inner')
neightracts.drop('index_right', axis=1, inplace=True)
fulllist = homes_clean.sjoin(neightracts, how='inner')
fulllist

,geometry,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid,index_right,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,LOGRECNO,name,listname,mapname,shape_leng,shape_area,cartodb_id,created_at,updated_at
0,POINT (-75.14337 40.00957),2022-05-24T00:00:00Z,D,415' N OF ERIE AVE,54230032,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,45.0,0.0,0.0,4,0.0,16.0,None,0.0,None,A,43,0,None,3753,4,3753 N DELHI ST,None,None,None,DELRAY BEACH FL,4899 NW 6TH STREET,33445,73800,None,1.0,3.0,NaN,2.0,1683.0,None,RJ SIMPLE SOLUTION LLC,None,432345900,E,C,2023-10-04T00:00:00Z,100N040379,2022-06-13T00:00:00Z,35000,None,None,None,FL,28040,ST,N,DELHI,None,59040.0,14760.0,F,720.0,960.0,H,None,None,None,I,1942,Y,19140,RM1,1001175031,24,ROW PORCH FRONT,395846194,22,23,42,101,019800,42101019800,198,Census Tract 198,G5020,S,541006,0,+40.0107245,-075.1421472,10523,HUNTING_PARK,Hunting Park,Hunting Park,32920.799360,3.902450e+07,73,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
345,POINT (-75.14088 40.00998),2022-05-24T00:00:00Z,None,119' N OF BUTLER ST,54140475,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,63.0,0.0,0.0,4,0.0,15.0,None,0.0,None,A,43,0,None,3814,4,3814 N FRANKLIN ST,None,None,None,ROSLYN PA,2905 JOYCE RD,19001,81600,None,1.0,3.0,NaN,2.0,1320.0,None,MANNERS MARIANA TR,None,432295700,E,C,2023-01-17T00:00:00Z,100N090149,2022-12-28T00:00:00Z,1,None,None,None,PA,34980,ST,N,FRANKLIN,None,65280.0,16320.0,F,945.0,1050.0,H,None,None,None,I,1940,Y,19140,RSA5,1001225372,24,ROW PORCH FRONT,395860759,22,23,42,101,019800,42101019800,198,Census Tract 198,G5020,S,541006,0,+40.0107245,-075.1421472,10523,HUNTING_PARK,Hunting Park,Hunting Park,32920.799360,3.902450e+07,73,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
390,POINT (-75.13708 40.00918),2022-05-24T00:00:00Z,A,132' W 5TH ST,54145195,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,43.0,0.0,0.0,4,0.0,13.0,None,0.0,None,A,43,0,None,517,3,517 W BUTLER ST,CALDERON PAULINO,None,None,PHILADELPHIA PA,12517 BISCAYNE DR,19154,59200,None,1.0,2.0,NaN,2.0,810.0,None,CALDERON PAULINO,CONCHA JORGE LUIS MEDINA,432220600,E,C,2023-01-31T00:00:00Z,100N8 18,2022-10-03T00:00:00Z,63500,None,None,None,PA,20080,ST,W,BUTLER,None,47360.0,11840.0,F,553.0,780.0,H,None,None,None,I,1940,Y,19140,RM1,1001112818,22,ROW TYPICAL,395859963,22,23,42,101,019800,42101019800,198,Census Tract 198,G5020,S,541006,0,+40.0107245,-075.1421472,10523,HUNTING_PARK,Hunting Park,Hunting Park,32920.799360,3.902450e+07,73,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
823,POINT (-75.13878 40.01089),2022-05-24T00:00:00Z,D,NEC OF PIKE ST,54132453,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,60.0,0.0,0.0,4,0.0,16.0,None,0.0,None,A,43,0,None,3901,4,3901 N MARSHALL ST,`3901 N MARSHALL STREET CORPORATION,None,None,PHILADELPHIA PA,3916 N 5TH ST,19140,108500,None,1.0,4.0,

### 2.4 Train a Random Forest on the sales data

In this step, you should follow the steps outlined in lecture to preprocess and train your model. We'll extend our analysis to do a hyperparameter grid search to find the best model configuration. As you train your model, follow the following steps:

**Preprocessing Requirements**
- Trim the sales data to those sales with prices between $3,000 and $1 million
- Set up a pipeline that includes both numerical columns and categorical columns
- Include one-hot encoded variable for the *neighborhood* of the sale, **instead of ZIP code**. We don't want to include multiple location based categories, since they encode much of the same information.

**Training requirements**
- Use a 70/30% training/test split and predict the log of the sales price.
- Use GridSearchCV to perform a k-fold cross validation that optimize *at least 2* hyperparameters of the RandomForestRegressor
- After fitting your model and finding the optimal hyperparameters, you should evaluate the score (R-squared) on the test set (the original 30% sample withheld)

**Note**: You don't need to include additional features (such as spatial distance features) or perform any extra feature engineering beyond what is required above to receive full credit. Of course, you are always welcome to experiment!

In [30]:
trimsales = fulllist[(fulllist['sale_price'] >= 3000) & (fulllist['sale_price'] <= 1000000)]
trimsales.head(2)

,geometry,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid,index_right,OBJECTID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,LOGRECNO,name,listname,mapname,shape_leng,shape_area,cartodb_id,created_at,updated_at
0,POINT (-75.14337 40.00957),2022-05-24T00:00:00Z,D,415' N OF ERIE AVE,54230032,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,45.0,0.0,0.0,4,0.0,16.0,None,0.0,None,A,43,0,None,3753,4,3753 N DELHI ST,None,None,None,DELRAY BEACH FL,4899 NW 6TH STREET,33445,73800,None,1.0,3.0,NaN,2.0,1683.0,None,RJ SIMPLE SOLUTION LLC,None,432345900,E,C,2023-10-04T00:00:00Z,100N040379,2022-06-13T00:00:00Z,35000,None,None,None,FL,28040,ST,N,DELHI,None,59040.0,14760.0,F,720.0,960.0,H,None,None,None,I,1942,Y,19140,RM1,1001175031,24,ROW PORCH FRONT,395846194,22,23,42,101,019800,42101019800,198,Census Tract 198,G5020,S,541006,0,+40.0107245,-075.1421472,10523,HUNTING_PARK,Hunting Park,Hunting Park,32920.79936,3.902450e+07,73,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00
390,POINT (-75.13708 40.00918),2022-05-24T00:00:00Z,A,132' W 5TH ST,54145195,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,198,N,None,None,43.0,0.0,0.0,4,0.0,13.0,None,0.0,None,A,43,0,None,517,3,517 W BUTLER ST,CALDERON PAULINO,None,None,PHILADELPHIA PA,12517 BISCAYNE DR,19154,59200,None,1.0,2.0,NaN,2.0,810.0,None,CALDERON PAULINO,CONCHA JORGE LUIS MEDINA,432220600,E,C,2023-01-31T00:00:00Z,100N8 18,2022-10-03T00:00:00Z,63500,None,None,None,PA,20080,ST,W,BUTLER,None,47360.0,11840.0,F,553.0,780.0,H,None,None,None,I,1940,Y,19140,RM1,1001112818,22,ROW TYPICAL,395859963,22,23,42,101,019800,42101019800,198,Census Tract 198,G5020,S,541006,0,+40.0107245,-075.1421472,10523,HUNTING_PARK,Hunting Park,Hunting Park,32920.79936,3.902450e+07,73,2013-03-19 17:41:50.508000+00:00,2013-03-19 17:41:50.743000+00:00


In [28]:
numerical_columns = trimsales.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = trimsales.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
    ('scaler', StandardScaler(), numerical_columns)
])
X = trimsales.drop('sale_price', axis=1)
y = trimsales['sale_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

model = LinearRegression()
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)

ValueError: A given column is not a column of the dataframe

### 2.5 Calculate the percent error of your model predictions for each sale in the test set

Fit your best model and use it to make predictions on the test set.

**Note:** This should be the percent error in terms of **sale price**. You'll need to convert if your model predicted the log of sales price!

### 2.6 Make a data frame with percent errors and census tract info for each sale in the test set

Create a data frame that has the property geometries, census tract data, and percent errors for all of the sales in the test set.

**Notes**

- When using the "train_test_split()" function, the index of the test data frame includes the labels from the original sales data frame
- You can use this index to slice out the test data from the original sales data frame, which should include the census tract info and geometries
- Add a new column to this data frame holding the percent error data
- Make sure to use the percent error and not the absolute percent error



### 2.8 Plot a map of the median percent error by census tract 

- You'll want to group your data frame of test sales by the `GEOID10` column and take the median of you percent error column
- Merge the census tract geometries back in and use geopandas to plot.

### 2.9 Compare the percent errors in Qualifying Census Tracts and other tracts 

[Qualifying Census Tracts](https://www.huduser.gov/portal/datasets/qct.html) are a poverty designation that HUD uses to allocate housing tax credits

- I've included a list of the census tract names that qualify in Philadelphia
- Add a new column to your dataframe of test set sales that is True/False depending on if the tract is a QCT
- Then, group by this new column and calculate the median percent error

**You should find that the algorithm's accuracy is significantly worse in these low-income, qualifying census tracts**

In [1]:
qct = ['5',
 '20',
 '22',
 '28.01',
 '30.01',
 '30.02',
 '31',
 '32',
 '33',
 '36',
 '37.01',
 '37.02',
 '39.01',
 '41.01',
 '41.02',
 '56',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '69',
 '70',
 '71.01',
 '71.02',
 '72',
 '73',
 '74',
 '77',
 '78',
 '80',
 '81.01',
 '81.02',
 '82',
 '83.01',
 '83.02',
 '84',
 '85',
 '86.01',
 '86.02',
 '87.01',
 '87.02',
 '88.01',
 '88.02',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '98.01',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '119',
 '121',
 '122.01',
 '122.03',
 '131',
 '132',
 '137',
 '138',
 '139',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '151.01',
 '151.02',
 '152',
 '153',
 '156',
 '157',
 '161',
 '162',
 '163',
 '164',
 '165',
 '167.01',
 '167.02',
 '168',
 '169.01',
 '169.02',
 '170',
 '171',
 '172.01',
 '172.02',
 '173',
 '174',
 '175',
 '176.01',
 '176.02',
 '177.01',
 '177.02',
 '178',
 '179',
 '180.02',
 '188',
 '190',
 '191',
 '192',
 '195.01',
 '195.02',
 '197',
 '198',
 '199',
 '200',
 '201.01',
 '201.02',
 '202',
 '203',
 '204',
 '205',
 '206',
 '208',
 '239',
 '240',
 '241',
 '242',
 '243',
 '244',
 '245',
 '246',
 '247',
 '249',
 '252',
 '253',
 '265',
 '267',
 '268',
 '271',
 '274.01',
 '274.02',
 '275',
 '276',
 '277',
 '278',
 '279.01',
 '279.02',
 '280',
 '281',
 '282',
 '283',
 '284',
 '285',
 '286',
 '287',
 '288',
 '289.01',
 '289.02',
 '290',
 '291',
 '293',
 '294',
 '298',
 '299',
 '300',
 '301',
 '302',
 '305.01',
 '305.02',
 '309',
 '311.01',
 '312',
 '313',
 '314.01',
 '314.02',
 '316',
 '318',
 '319',
 '321',
 '325',
 '329',
 '330',
 '337.01',
 '345.01',
 '357.01',
 '376',
 '377',
 '380',
 '381',
 '382',
 '383',
 '389',
 '390']